In [21]:
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv('GROQ_API_KEY')

In [22]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key=API_KEY,
    model_name="llama-3.1-70b-versatile"

)

response = llm.invoke('The first person to land on moon was ...')
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [23]:
# from groq import Groq

# # Initialize client
# client = Groq()

# # Properly formatted messages
# messages = [
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "First person to land on the moon"}
# ]

# # Generate completion
# completion = client.chat.completions.create(
#     model="llama-3.1-70b-versatile",
#     messages=messages,
#     temperature=1,
#     max_tokens=1024,
#     top_p=1,
#     stream=True,
#     stop=None,
# )

# # Stream the response
# for chunk in completion:
#     print(chunk.choices[0].delta.content or "", end="")


In [1]:
import chromadb

client  = chromadb.Client()
collection = client.create_collection(name="my_collection")

In [2]:
collection.add(
    documents=[
        "This docuemnt is about New York",
        "This document is about Delhi"

    ],
    ids=['id1', 'id2']

)

In [26]:
all_docs = collection.get()
doc = collection.get(ids='id1')
doc

{'ids': ['id1'],
 'embeddings': None,
 'documents': ['This docuemnt is about New York'],
 'uris': None,
 'data': None,
 'metadatas': [None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [9]:
# content which is more similar and related to the query asked will be showed first in documents.
#Try using "Query is about AIR Pollution" or "Query is baout brooklyn bridge"

# Completely based on SEMANTIC SEARCHING
results = collection.query(
    query_texts=['Query is about pstatue of kidnaping'],
    n_results=2
)
results

{'ids': [['id2', 'id1']],
 'embeddings': None,
 'documents': [['This document is about Delhi',
   'This docuemnt is about New York']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.6649165153503418, 1.674971580505371]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [28]:
# To Delete
collection.delete(ids=all_docs['ids'])
collection.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [41]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.ey.com/ey/job/Coimbatore-EY-GDS-Consulting-AI-and-DATA-Python-Data-Eng-Senior-TN-641014/1144196001")
page_data = loader.load().pop().page_content
print(page_data)














EY-GDS Consulting-AI and DATA-Python Data Eng-Senior Job Details | EY






























Cookie information
Welcome to the EY careers job search site. This website is based on the SuccessFactors software provided by SAP. On this page, functional and optional cookies are used to improve your experience and design our careers site more user-friendly and in line with your needs. In this context, cookies from providers in third countries may also be used and data may be transmitted to providers such as social media services outside the EU. For this we require your consent. By clicking "Accept All Cookies", you agree to these. This also includes your consent to the transmission of certain personal data to third countries, including the USA, in accordance with Art. 49 (1) (a) GDPR. You can select your settings by clicking “Modify Cookie Preferences” to confirm your choices from the optional cookie tracking and selecting the required cookies required to remain on the s

In [42]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPPED TEST FROM WEBSITE:
    {page_data}
    ### INSTRUCTION 
    The scrapped text is from the career's page of a website.
    Yor job is to extract the job postings and return them in JSONO fromat containing following keys: `role`, `expereience`, `skills`, and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):"""

)

chain_extract = prompt_extract | llm | JsonOutputParser()
res = chain_extract.invoke(input={'page_data':page_data})
print(res)
# res = json.dumps(res, indent=4)
# print(type(res))

[{'role': 'Senior Data Engineer', 'experience': '5+ years', 'skills': ['Python', 'Distributed Data Technologies (e.g., Pyspark)', 'Cloud platform deployment and tools (e.g., Kubernetes)', 'Relational SQL databases', 'DevOps and continuous integration', 'AWS cloud services and technologies (i.e., Lambda, S3, DMS, Step Functions, Event Bridge, Cloud Watch, RDS)', 'Databricks/ETL', 'IICS/DMS', 'GitHub', 'Event Bridge, Tidal'], 'description': 'Develop and maintain scalable data pipelines, in line with ETL principles, and build out new integrations, using AWS native technologies, to support continuing increases in data source, volume, and complexity. Define data requirements, gather, and mine data, while validating the efficiency of data tools in the Big Data Environment. Lead the evaluation, implementation and deployment of emerging tools and processes to improve productivity. Implement processes and systems to provide accurate and available data to key stakeholders, downstream systems, an

In [37]:
import csv

# Data to write to the CSV
data = [
    ["Techstack", "Links"],
    ["React, Node.js, MongoDB", "https://example.com/react-portfolio"],
    ["Angular, .NET, SQL Server", "https://example.com/angular-portfolio"],
    ["Vue.js, Ruby on Rails, PostgreSQL", "https://example.com/vue-portfolio"],
    ["Python, Django, MySQL", "https://example.com/python-portfolio"],
    ["Java, Spring Boot, Oracle", "https://example.com/java-portfolio"],
    ["Flutter, Firebase, Google Cloud", "https://example.com/flutter-portfolio"],
    ["WordPress, PHP, MySQL", "https://example.com/wordpress-portfolio"],
    ["Magento, PHP, MySQL", "https://example.com/magento-portfolio"],
    ["React Native, Node.js, MongoDB", "https://example.com/react-native-portfolio"],
    ["iOS, Swift, Core Data", "https://example.com/ios-portfolio"],
    ["Android, Java, Room Persistence", "https://example.com/android-portfolio"],
    ["Kotlin, Android, Firebase", "https://example.com/kotlin-portfolio"],
    ["Android TV, Kotlin, Android NDK", "https://example.com/android-tv-portfolio"],
    ["iOS, Swift, ARKit", "https://example.com/ios-ar-portfolio"],
    ["Cross-platform, Xamarin, Azure", "https://example.com/xamarin-portfolio"],
    ["Backend, Kotlin, Spring Boot", "https://example.com/kotlin-backend-portfolio"],
    ["Frontend, TypeScript, Angular", "https://example.com/typescript-frontend-portfolio"],
    ["Full-stack, JavaScript, Express.js", "https://example.com/fullstack-portfolio"],
    ["Machine Learning, Python, TensorFlow", "https://example.com/ml-python-portfolio"],
    ["DevOps, Jenkins, Docker", "https://example.com/devops-portfolio"]
]

# Write to a CSV file
csv_file = "techstack_links.csv"
with open(csv_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"CSV file '{csv_file}' created successfully.")


PermissionError: [Errno 13] Permission denied: 'techstack_links.csv'

In [ ]:
import csv

# Data to write to the CSV
data = [
    ["Techstack", "Links"],
    ["Python, Flask, Django", "https://vinit-lathiwala.vercel.app/"],
    ["Machine Learning, Python, TensorFlow", "https://github.com/Vinitv38/Covid-19-detection-using-Deep-Learning"],
    ["AI Chatbot, Flask, FastAPI", "https://github.com/Vinitv38/Food-ChatBot"],
    ["Frontend, HTML5, CSS3, JavaScript", None],
    ["Backend, Python, Django, Flask", None],
    ["DevOps, AWS (Lambda, S3), Docker", None],
    ["Stock Analysis, Yahoo Finance API, AI", "https://github.com/Vinitv38/StockBot-Advisor"],
    ["GitHub Profile", "https://github.com/Vinitv38"],
    ["LinkedIn Profile", "http://www.linkedin.com/in/vinit-lathiwala/"],
    ["Portfolio", "https://vinit-lathiwala.vercel.app/"]
]

# Write to a CSV file
csv_file = "portfolio.csv"
with open(csv_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"CSV file '{csv_file}' created successfully.")


CSV file 'portfolio.csv' created successfully.


In [3]:
import pandas as pd
df = pd.read_csv('portfolio.csv')
df.head()

,Techstack,Links
0,"Python, Flask, Django",https://vinit-lathiwala.vercel.app/
1,"Machine Learning, Python, TensorFlow",https://github.com/Vinitv38/Covid-19-detection...
2,"AI Chatbot, Flask, FastAPI",https://github.com/Vinitv38/Food-ChatBot
3,"Frontend, HTML5, CSS3, JavaScript",NaN
4,"Backend, Python, Django, Flask",NaN


In [38]:
# Trial Purpose

import chromadb
import uuid

client = chromadb.PersistentClient('Trial_vectorstore')
collection = client.get_or_create_collection(name='techstack_links')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids = [str(uuid.uuid4())])

In [ ]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids = [str(uuid.uuid4())])

In [40]:
collection.get()

{'ids': ['b73ff6fd-297f-4c24-bca3-b80ae90032ac',
  '077acb9d-89c4-4fd3-9f44-1a5fb0f8684a',
  'af079073-ce0a-4733-8e77-f810de02fddc',
  '06e28b8d-962f-48fb-9af9-117cb58178f5',
  '67516913-18f3-4eae-9bab-47bd72b862a8',
  'c0b126af-ddbb-43ae-8ba5-74f82da741d8',
  'dc1d0739-0ac5-4778-a7bc-595082bde5e0',
  '8fedf47d-73d6-4806-b783-ae3420b8683e',
  'd4ef255f-be36-4282-9ca6-b78b89bfcf59',
  '3df0a3b4-50b7-471c-9d9f-57cf1d236893'],
 'embeddings': None,
 'documents': ['Python, Flask, Django',
  'Machine Learning, Python, TensorFlow',
  'AI Chatbot, Flask, FastAPI',
  'Frontend, HTML5, CSS3, JavaScript',
  'Backend, Python, Django, Flask',
  'DevOps, AWS (Lambda, S3), Docker',
  'Stock Analysis, Yahoo Finance API, AI',
  'GitHub Profile',
  'LinkedIn Profile',
  'Portfolio'],
 'uris': None,
 'data': None,
 'metadatas': [{'links': 'https://vinit-lathiwala.vercel.app/'},
  {'links': 'https://github.com/Vinitv38/Covid-19-detection-using-Deep-Learning'},
  {'links': 'https://github.com/Vinitv38/Food

In [49]:
links = collection.query(query_texts=res['skills'],n_results=2).get('metadatas')
links

TypeError: list indices must be integers or slices, not str

In [48]:
res['skills']

TypeError: list indices must be integers or slices, not str

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.livemint.com/market/stock-market-news")
page_data = loader.load().pop()
print(page_data)

page_content='

          




Stock Market News: Latest Stock News Today, Stock Market LIVE, Stock Market Headline Updates | Mint



















 



 







          Explore          Sign in      e-paper Subscribe  Sign In     Thursday, 28 November 2024              Stocks       Mutual Funds       News                     Home News Markets Premium Money Mutual Fund Personal Loan Companies Technology Web Stories In Charts Opinion Videos      All Companies  Technology Markets Money MyMint Mutual Funds Insurance Auto  Industry  Personal Finance        Hello User  Sign in      Sign Out     My Account My Account Subscribe  My Watchlist   Newsletters   Notifications      My Reads   For You   View Less -  View More +   Data Insights   Market Dashboard   Bullion      Gold   Silver     Fuel      Petrol   Diesel     Commodities      Gold   GoldM   Aluminum   Menthaoil   Silver   SilverMIC   GoldPetal   Natural Gas   Copper   Zinc   SilverM   CrudeOil   GoldGinuea   Lead        CryptoCurr

In [5]:
from langchain.document_loaders import WebBaseLoader

# Specify the URL
url = "https://www.livemint.com/markets"  # Use the base URL from the uploaded image

# Use the WebBaseLoader
loader = WebBaseLoader(url, follow_links=True, max_depth=1)

# Load the content
docs = loader.load()

# Display content
for doc in docs:
    print(f"Title: {doc.metadata.get('title')}")
    print(f"Content: {doc.page_content[:500]}...")  # Limit for preview


TypeError: WebBaseLoader.__init__() got an unexpected keyword argument 'follow_links'